In [1]:
import os

In [2]:
%pwd

'D:\\Desktop\\Superteams AI\\Task 1 Steps to Monitoring DSPy-Qdrant Powered RAG with Prometheus or Grafana\\RAG-Stack-using-DSPy-Qdrant-Mistral-7B\\Research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'D:\\Desktop\\Superteams AI\\Task 1 Steps to Monitoring DSPy-Qdrant Powered RAG with Prometheus or Grafana\\RAG-Stack-using-DSPy-Qdrant-Mistral-7B'

In [5]:
import logging
from pathlib import Path

In [6]:
# Configure the logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [7]:
import pyarrow.parquet as pq

# Read the Parquet file into a pyarrow Table
table = pq.read_table("Dataset/train-00000-of-00001-a77e2814210655f1.parquet")

# Convert the Table to a pandas DataFrame
df = table.to_pandas()

# Now you can work with the DataFrame as usual
df.head()


,instruction,input,output,__index_level_0__
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in...",137213
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...,67540
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...,109854
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...,137309
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ...",135382


In [8]:
# Create dfcopy with 100 rows
dfcopy = df.head(15).copy()

# Display the first few rows
dfcopy.head()

,instruction,input,output,__index_level_0__
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in...",137213
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...,67540
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...,109854
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...,137309
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ...",135382


In [9]:
dfcopy = dfcopy.drop("__index_level_0__", axis=1)
dfcopy.head()

,instruction,input,output
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in..."
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ..."


In [10]:
# from transformers import AutoTokenizer, AutoModel
# from pathlib import Path

# def download_model_and_tokenizer(model_name, save_path):
#     """
#     Download and save both the model and the tokenizer to the specified directory.

#     Parameters:
#         model_name (str): Name of the model to download.
#         save_path (str or Path): Path to the directory where the model and tokenizer will be saved.
#     """
#     # Create the save path if it doesn't exist
#     save_path = Path(save_path)
#     save_path.mkdir(parents=True, exist_ok=True)
    
#     # Initialize tokenizer and model
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModel.from_pretrained(model_name)
    
#     # Save tokenizer
#     tokenizer.save_pretrained(save_path)
    
#     # Save model
#     model.save_pretrained(save_path)

# # Example usage
# model_name = 'sentence-transformers/all-MiniLM-L12-v2'  # Model name to download
# save_path = Path("MiniLM-L12-v2/")  # Path where model and tokenizer will be saved
# download_model_and_tokenizer(model_name, save_path)


In [11]:
from transformers import AutoTokenizer, AutoModel

def load_model_and_tokenizer(model_path):
    """
    Load the model and tokenizer from the specified directory.

    Parameters:
        model_path (str or Path): Path to the directory containing the saved model and tokenizer.

    Returns:
        tokenizer (transformers.PreTrainedTokenizer): Loaded tokenizer.
        model (transformers.PreTrainedModel): Loaded model.
    """
    model_path = Path(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path)
    return tokenizer, model

# Load the model and tokenizer
model_path = Path("MiniLM-L12-v2/")
tokenizer, model = load_model_and_tokenizer(model_path)

D:\Desktop\Superteams AI\Task 1 Steps to Monitoring DSPy-Qdrant Powered RAG with Prometheus or Grafana\RAG-Stack-using-DSPy-Qdrant-Mistral-7B\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
import torch

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def generate_embedding(text):
    # Tokenize input text
    encoded_input = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    # Compute token embeddings with model
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Perform mean pooling
    sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])
    # Convert to numpy for FAISS compatibility and ensure it's 2D
    return sentence_embedding.cpu().numpy().reshape(1, -1)

In [13]:
dfcopy['merged_data'] = dfcopy.apply(lambda row: row['input'] + ' ' + row['output'], axis=1)
dfcopy.head()

,instruction,input,output,merged_data
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in...",hi. im a home health aide and i have a client ...
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...,RATIONALE: The COVID-19 pandemic struck an imm...
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...,Objectives: To investigate the experience of p...
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...,"Hi, Im sorry to bother you but I have fpund a ..."
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ...","Hi, my penis has a slightly lighter spot at t..."


In [14]:
# Generate embeddings for the 'input' column
dfcopy['embeddings'] = dfcopy['output'].apply(lambda x: generate_embedding(x)[0].tolist())
dfcopy.head()

,instruction,input,output,merged_data,embeddings
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in...",hi. im a home health aide and i have a client ...,"[-0.2175416499376297, -0.13464151322841644, 0...."
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...,RATIONALE: The COVID-19 pandemic struck an imm...,"[-0.05913766846060753, -0.06794142723083496, -..."
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...,Objectives: To investigate the experience of p...,"[0.05637330189347267, 0.07810498774051666, 0.0..."
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...,"Hi, Im sorry to bother you but I have fpund a ...","[0.01962941884994507, -0.1911410689353943, -0...."
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ...","Hi, my penis has a slightly lighter spot at t...","[-0.055171988904476166, 0.012183529324829578, ..."


In [15]:
df.head()

,instruction,input,output,__index_level_0__
0,"If you are a doctor, please answer the medical...",hi. im a home health aide and i have a client ...,"hi, thanks for contacting chatbot. swelling in...",137213
1,Please summerize the given abstract to a title,RATIONALE: The COVID-19 pandemic struck an imm...,Hydroxychloroquine vs. Azithromycin for Hospit...,67540
2,Please summerize the given abstract to a title,Objectives: To investigate the experience of p...,Playing the harmonica with chronic obstructive...,109854
3,"If you are a doctor, please answer the medical...","Hi, Im sorry to bother you but I have fpund a ...",welcome to chatbot .1. the history suggest a p...,137309
4,"If you are a doctor, please answer the medical...","Hi, my penis has a slightly lighter spot at t...","hello, skin color changes, discoloration etc. ...",135382


In [16]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

# Initialize Qdrant client
qdrant_client = QdrantClient(host='localhost', port=6333)
collection_name = "medical_dataset"

# Specify the vectors' configuration
vectors_config = VectorParams(
    size=model.config.hidden_size,  # The size of your embeddings
    distance=Distance.COSINE  # The distance metric for the vector space
)

# Create or recreate the collection with the specified configuration
qdrant_client.recreate_collection(
    collection_name=collection_name,
    vectors_config=vectors_config,
    # Optionally, you can specify other parameters for the collection
)



INFO:httpx:HTTP Request: DELETE http://localhost:6333/collections/medical_dataset "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset "HTTP/1.1 200 OK"


True

In [17]:
 # Insert data into Qdrant
for index, row in dfcopy.iterrows():
    qdrant_client.upsert(
        collection_name=collection_name,
        points=[{
            "id": index,  # Using the dataframe index as the ID
            "vector": row['embeddings'],
            "payload": {
                # "instruction": row['instruction'], 
                # "input": row['input'],
                "output": row['output']
            }
        }]
    )


INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wait=true "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: PUT http://localhost:6333/collections/medical_dataset/points?wa

In [18]:
logger.info(f"qdrant '{collection_name}' collection created")

INFO:__main__:qdrant 'medical_dataset' collection created


In [19]:
from langchain.vectorstores import Qdrant

url = "http://localhost:6333"  # URL where the Qdrant service is running
collection_name =  "medical_dataset"  # Name of the collection in Qdrant

# Initialize the Qdrant client with the specified URL
client = QdrantClient(
    url=url,
    prefer_grpc=False  # Indicates whether to use gRPC for communication
)

logger.info(f"QdrantClient initialized: {client}")  # Prints the client information
logger.info(f"#################################")  # Prints a separator line

# Create a Qdrant object with the specified client, embeddings, and collection name
# Initialize the Qdrant vector store from langchain
db = Qdrant(
    client=client,
    embeddings=dfcopy['embeddings'].tolist(),  # Use the generated embeddings
    collection_name=collection_name
)

logger.info(f"Qdrant vector store initialized: {db}") # Prints the database object information

INFO:__main__:QdrantClient initialized: <qdrant_client.qdrant_client.QdrantClient object at 0x000002C6EA5DA8C0>
INFO:__main__:#################################
D:\Desktop\Superteams AI\Task 1 Steps to Monitoring DSPy-Qdrant Powered RAG with Prometheus or Grafana\RAG-Stack-using-DSPy-Qdrant-Mistral-7B\venv\lib\site-packages\langchain_community\vectorstores\qdrant.py:150: UserWarning: `embeddings` should be an instance of `Embeddings`.Using `embeddings` as `embedding_function` which is deprecated
  warnings.warn(
INFO:__main__:Qdrant vector store initialized: <langchain_community.vectorstores.qdrant.Qdrant object at 0x000002C698357430>


In [20]:
def similarity_search_with_score(query, k=5):
    query_embedding = generate_embedding(query)[0].tolist()
    search_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=k,
        with_payload=True,
        with_vectors=False
    )
    return search_results

query = "What is Azithromycin?"
search_results = similarity_search_with_score(query=query, k=5)

for result in search_results:
    doc_id = result.id
    score = result.score
    payload = result.payload  # The payload should contain your text or a reference to it.

    # Assuming the payload contains a field 'input' where the text is stored
    doc_content = payload.get('output', 'No content available')

    # Print the similarity score and document content
    logger.info({"score": score, "doc_id": doc_id, "content": doc_content})

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/medical_dataset/points/search "HTTP/1.1 200 OK"
INFO:__main__:{'score': 0.4070936, 'doc_id': 1, 'content': 'Hydroxychloroquine vs. Azithromycin for Hospitalized Patients with COVID-19 (HAHPS): Results of a Randomized, Active Comparator Trial'}
INFO:__main__:{'score': 0.26959538, 'doc_id': 11, 'content': 'hello there... gum infections occur because of various factors... herpes infection being one amongst them.... people with herpes infection have a decreased immunity.... acyclovir given would further compromise the status... that is when the infections set in... apart from the immunity, local factors also play an important role in the gum hygiene... people with oral sexual behavior (multiple partners) tend to have more of these infections.... you require oral prophylaxis with appropriate antibiotics... please use betadine rinses (5ml) 3-4 times daily to prevent further bacterial growth.... consult you dental surgeon for fur

In [21]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp

model_path = r"Mistral7b_Model/mistral-7b-instruct-v0.1.Q5_K_M.gguf"

n_gpu_layers = -1  # Use GPU if possible, rest on CPU
n_batch = 512  # Adjust based on GPU VRAM

try:
    llm = LlamaCpp(
        model_path=model_path,  # Only use model_url, not model_path
        n_gpu_layers=n_gpu_layers,
        n_batch=n_batch,
        verbose=True  # Needed for callback manager
    )

    # Add your code using llm here
except Exception as e:
    print(f"Error creating LlamaCpp instance: {e}")

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from Mistral7b_Model/mistral-7b-instruct-v0.1.Q5_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:           

In [23]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate.from_template(template)

llm_chain = LLMChain(prompt=prompt, llm=llm)
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"
llm_chain.run(question)

Llama.generate: prefix-match hit

llama_print_timings:        load time =     519.76 ms
llama_print_timings:      sample time =      63.06 ms /   113 runs   (    0.56 ms per token,  1791.94 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    4259.56 ms /   113 runs   (   37.70 ms per token,    26.53 tokens per second)
llama_print_timings:       total time =    5218.34 ms /   114 tokens


" \n\nFirst, let's figure out the year Justin Bieber was born. According to his official bio, that would be March 1st, 1994.\n\nNow, we can use that information to determine which NFL team won the Super Bowl that year. The Super Bowl XVIII took place on January 30th, 1994. So, the winning team must be the one that played on that day.\n\nThe Super Bowl XVIII winning team was the Dallas Cowboys!"

In [24]:
from langchain import PromptTemplate, LLMChain


template = '''[INST]: You are a Medical expert analyst bot, below presents a context from which the a question will be asked, give your valuable insights as well.[\INST]\n
Context: {context}.\n
Question: {question}\n
Answer: '''

In [25]:
rag_prompt = PromptTemplate(template=template, input_variables=["context","question"])


callbacks = [StreamingStdOutCallbackHandler()]
llm_chain = LLMChain(prompt=rag_prompt, llm=llm, verbose=True)

In [26]:
def similarity_search_with_score(query, k=5):
    query_embedding = generate_embedding(query)[0].tolist()
    search_results = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_embedding,
        limit=k,
        with_payload=True,
        with_vectors=False
    )

    # Extract the document content from the payload and include it in the results
    results_with_content = []
    for result in search_results:
        doc_id = result.id
        score = result.score
        payload = result.payload  # The payload should contain your text or a reference to it.
        
        # Extract the document content from the payload
        doc_content = payload.get('output', 'No content available')

        results_with_content.append((score, doc_content))

    # Sort the results based on the similarity score in descending order
    sorted_results = sorted(results_with_content, key=lambda x: x[0], reverse=True)

    # Concatenate the content of the top k results
    concatenated_content = ' '.join([content for _, content in sorted_results[:k]])

    return concatenated_content

# Query for which you want to find similar documents
# query = "What is Azithromycin?"
# concatenated_contents = similarity_search_with_score(query=query, k=2)

# # Print the concatenated content of the top documents
# print("Concatenated content of the top documents:", concatenated_contents)


In [27]:
query = "What is Azithromycin ?"
resp = llm_chain.invoke(
    input={"question":query,
           "context": similarity_search_with_score(query,k=5)
          }
)
print(resp['text'])

INFO:httpx:HTTP Request: POST http://localhost:6333/collections/medical_dataset/points/search "HTTP/1.1 200 OK"
Llama.generate: prefix-match hit




> Entering new LLMChain chain...
Prompt after formatting:
[INST]: You are a Medical expert analyst bot, below presents a context from which the a question will be asked, give your valuable insights as well.[\INST]

Context: Hydroxychloroquine vs. Azithromycin for Hospitalized Patients with COVID-19 (HAHPS): Results of a Randomized, Active Comparator Trial hello there... gum infections occur because of various factors... herpes infection being one amongst them.... people with herpes infection have a decreased immunity.... acyclovir given would further compromise the status... that is when the infections set in... apart from the immunity, local factors also play an important role in the gum hygiene... people with oral sexual behavior (multiple partners) tend to have more of these infections.... you require oral prophylaxis with appropriate antibiotics... please use betadine rinses (5ml) 3-4 times daily to prevent further bacterial growth.... consult you dental surgeon for further clini


llama_print_timings:        load time =     519.76 ms
llama_print_timings:      sample time =      79.49 ms /   145 runs   (    0.55 ms per token,  1824.11 tokens per second)
llama_print_timings: prompt eval time =     540.66 ms /   299 tokens (    1.81 ms per token,   553.03 tokens per second)
llama_print_timings:        eval time =    4658.96 ms /   144 runs   (   32.35 ms per token,    30.91 tokens per second)
llama_print_timings:       total time =    6403.56 ms /   443 tokens



> Finished chain.

Azithromycin is an antibiotic used to treat bacterial infections caused by certain types of bacteria such as Mycobacterium tuberculosis and Helicobacter pylori. It works by inhibiting bacterial DNA replication and blocking protein synthesis in the bacteria. In addition to its antibacterial properties, Azithromycin also has anti-inflammatory and immunomodulatory effects, making it useful in treating various infectious diseases such as pneumonia and skin infections. However, its use is limited due to its potential interactions with other medications and its potential side effects such as gastrointestinal upset and liver damage.
